In [14]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers import Dense, Input
import numpy as np
import pandas as pd
import os
import sys

In [15]:
def get_dataset():
  data = pd.read_csv("/home/Dataset JST Multi Layer Perceptron.csv")
  X = data[['IR520nm', 'IR680nm', 'IR710nm', 'IR740nm', 'IR770nm', 'IR800nm', 'IR830nm', 'IR880nm']]
  Y = data[['Kadar_minyak', 'ALB']]
  return X, Y

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X, Y = get_dataset()
Y_scaler = StandardScaler().fit(Y)
Y = Y_scaler.transform(Y)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [17]:
def build_model():
    input_layer = Input(shape=X_train.shape[1:])
    first_dense = Dense(units='512', activation='relu')(input_layer)
    second_dense = Dense(units='256', activation='relu')(first_dense)
    third_dense = Dense(units='128', activation='relu')(second_dense)
    y1_output = Dense(units='1', name='minyak')(third_dense)
    fourth_dense = Dense(units='512', activation='relu')(input_layer)
    fifth_dense = Dense(units='256', activation='relu')(fourth_dense)
    sixth_dense = Dense(units='128', activation='relu')(fifth_dense)
    y2_output = Dense(units=1, name='alb')(sixth_dense)
    model = Model(inputs=input_layer, outputs=[y1_output, y2_output])
    return model

In [18]:
model = build_model()
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss={'minyak': 'mean_squared_error', 'alb': 'mean_squared_error'})

In [19]:
history = model.fit(X_train, Y_train, epochs=1000, batch_size=169, validation_data=(X_test, Y_test))

Epoch 1/1000
1/1 [==============================] - 1s 915ms/step - loss: 1.9942 - minyak_loss: 0.9894 - alb_loss: 1.0048 - val_loss: 2.0541 - val_minyak_loss: 1.0237 - val_alb_loss: 1.0304
Epoch 2/1000
1/1 [==============================] - 0s 42ms/step - loss: 1.9697 - minyak_loss: 0.9788 - alb_loss: 0.9909 - val_loss: 2.0393 - val_minyak_loss: 1.0099 - val_alb_loss: 1.0294
Epoch 3/1000
1/1 [==============================] - 0s 40ms/step - loss: 1.9571 - minyak_loss: 0.9714 - alb_loss: 0.9857 - val_loss: 2.0349 - val_minyak_loss: 1.0156 - val_alb_loss: 1.0193
Epoch 4/1000
1/1 [==============================] - 0s 43ms/step - loss: 1.9419 - minyak_loss: 0.9620 - alb_loss: 0.9799 - val_loss: 2.0292 - val_minyak_loss: 1.0153 - val_alb_loss: 1.0139
Epoch 5/1000
1/1 [==============================] - 0s 44ms/step - loss: 1.9288 - minyak_loss: 0.9531 - alb_loss: 0.9757 - val_loss: 2.0195 - val_minyak_loss: 1.0071 - val_alb_loss: 1.0125
Epoch 6/1000
1/1 [==============================] - 0s

In [42]:
def report_result(X, Y):
  Y = pd.DataFrame(Y)
  hasil = model.predict(X)
  minyak_result = hasil[0]
  alb_result = hasil[1]
  prediksi_array = []

  for index, item in enumerate(minyak_result):
    prediksi_array.append([item[0], alb_result[index][0]])

  prediksi = np.array(prediksi_array)
  prediksi = pd.DataFrame(Y_scaler.inverse_transform(prediksi))

  errors_minyak = []
  absolute_error_minyak = []
  tingkat_error_minyak = []
  errors_alb = []
  absolute_error_alb = []
  tingkat_error_alb = []

  for index, item in enumerate(prediksi[0]):
    minyak_asli = Y['Kadar_minyak'][index]
    minyak_prediksi = prediksi[0][index]
    errors_minyak.append(round(minyak_asli - minyak_prediksi))
    absolute = f'{abs(minyak_asli - minyak_prediksi):.2f}'
    absolute_error_minyak.append(absolute)
    tingkat_error_minyak.append(f'{(abs(minyak_asli - minyak_prediksi) / minyak_asli):.2f}')
    
    alb_asli = Y['ALB'][index]
    alb_prediksi = prediksi[1][index]
    errors_alb.append(round(alb_asli - alb_prediksi))
    absolute = f'{abs(alb_asli - alb_prediksi):.2f}'
    absolute_error_alb.append(absolute)
    tingkat_error_alb.append(f'{(abs(alb_asli - alb_prediksi) / alb_asli):.2f}')

  X.insert(8, 'Kadar Minyak', Y[['Kadar_minyak']])
  X.insert(9, 'Prediksi Minyak', prediksi[[0]])
  X.insert(10, 'Error Minyak', errors_minyak)
  X.insert(11, 'Absolute Error Minyak', absolute_error_minyak)
  X.insert(12, 'Absolute/Minyak', tingkat_error_minyak)
  X.insert(13, 'ALB', Y[['ALB']])
  X.insert(14, 'Prediksi ALB', prediksi[[1]])
  X.insert(15, 'Error ALB', errors_alb)
  X.insert(16, 'Absolute Error ALB', absolute_error_alb)
  X.insert(17, 'Absolute/ALB', tingkat_error_alb)
  X.index = np.arange(1, len(X) + 1)
  X.to_excel('/home/Hasil Prediksi Multi-Output 8 Spektrum.xlsx')

In [21]:
def predict(X):
  return model.predict(X)

In [41]:
X, Y = get_dataset()
# report_result(X, Y)
prediction = predict([[0.51, 0.21, 0.58, 0.22, 0.41, 0.314, 0.213, 0.23]])
prediction

     Kadar_minyak    ALB
0           9.312  12.97
1           9.312  12.97
2          17.308   6.86
3          17.308   6.86
4          23.355  10.50
..            ...    ...
207        15.395   8.55
208        20.049   7.34
209        20.049   7.34
210        17.115  12.36
211        17.115  12.36

[212 rows x 2 columns]
7/7 [==============================] - 0s 3ms/step


TypeError: ignored

In [36]:
model.save('/content/model.h5')